In [1]:
%matplotlib inline

# Guardar y cargar el modelo - 06 Cargar el zip

## Cargar el modelo

Para cargar el modelo, no necesitamos crear nada del modelo, con importarlo basta.

In [2]:
import torch
from torch import nn

path = "data/modelo.zip"
model_scratch = torch.jit.load(path)

## Evaluar el modelo cargado

Nos descargamos el dataset de test y evaluamos el modelo

In [3]:
from torchvision import datasets
from torchvision.transforms import ToTensor

dataset_test = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

Files already downloaded and verified


Se crea el dataloader

In [4]:
from torch.utils.data import DataLoader

BATCH_SIZE = 64

test_dataloader = DataLoader(dataset_test, batch_size=BATCH_SIZE)

Se crea el bucle de test

In [5]:
import torch

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            # X and y to device
            X, y = X.to(device), y.to(device)
            
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Se lleva la red a la GPU, si la hay

In [6]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

model_scratch.to(device)

Using cuda device


RecursiveScriptModule(
  original_name=NeuralNetworkFromScratch
  (flatten): RecursiveScriptModule(original_name=Flatten)
  (linear_relu_stack): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(original_name=Linear)
    (1): RecursiveScriptModule(original_name=ReLU)
    (2): RecursiveScriptModule(original_name=Linear)
    (3): RecursiveScriptModule(original_name=ReLU)
    (4): RecursiveScriptModule(original_name=Linear)
  )
)

Se crea una función de pérdida

In [7]:
loss_fn = nn.CrossEntropyLoss()

Se comprueba el desempeño de la red en el dataset de test

In [8]:
test_loop(test_dataloader, model_scratch, loss_fn)

Test Error: 
 Accuracy: 38.5%, Avg loss: 1.704597 



Obtenemos métricas similares a cuando creamos y entrenamos la red